Import packages

In [ ]:
import numpy as np
import pandas as pd
import pymcdm
import pysensmcda
from pymcdm.methods import COMET, TOPSIS, MARCOS
from pymcdm.methods.comet_tools import MethodExpert
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects

Read data

In [ ]:
data_df = pd.read_csv('data_2022.csv', index_col=0)

Show data

In [ ]:
data_df

Abbreviations for countries name

In [ ]:
country_abbr = ['BE', 'HR', 'EE', 'FI', 'FR', 'DE', 'GR', 'HU', 'LT', 'NL', 'NO', 'PL', 'SK', 'ES', 'SE', 'CH']

Split data into four submodels

In [ ]:
columns = np.array(data_df.columns)
columns_p11 = columns[2:11]
columns_p12 = columns[11:20]
columns_p21 = columns[20:29]
columns_p22 = columns[29:]

Evaluate submodels with MARCOS method

In [ ]:
criteria_types = np.array([-1, -1, -1, 1, 1, -1, 1, 1, 1])

ranks = []

for matrix_columns in [columns_p11, columns_p12, columns_p21, columns_p22]:
    matrix = data_df[matrix_columns].to_numpy()
    
    weights = pymcdm.weights.equal_weights(matrix)
    marcos = MARCOS(normalization_function=pymcdm.normalizations.max_normalization)
    prefs = marcos(matrix, weights, criteria_types)
    ranking = marcos.rank(prefs)

    ranks.append(list(ranking))

ranks = np.array(ranks)
initial_ranks = ranks

Ranking visualization method

In [ ]:
# modified version of method from pymcdm
# https://gitlab.com/shekhand/mcda/-/blob/master/pymcdm/visuals/ranking_flows.py

def ranking_flows(rankings,
                  labels=None,
                  colors=None,
                  alt_indices=None,
                  space=0.1,
                  main_plot_kwargs=dict(),
                  missing_plot_kwargs=dict(),
                  marker_plot_kwargs=dict(),
                  better_grid=False,
                  ax=None):
    if ax is None:
        ax = plt.gca()

    rankings = np.array(rankings)

    # If colors list is not provided, use default matplotlib colors
    if colors is None:
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    # color_picker[i] will determine which color from colors list should be
    # chosen for this alternative. This way, color cycle will be ordered by
    # positions in the first rankings and not by order of alternatives.
    color_picker = np.zeros(rankings.shape[1], dtype='int')
    color_picker[np.argsort(rankings[0])] = np.arange(rankings.shape[1],
                                                      dtype='int')
    # Ensure that rankings is array
    rankings = np.array(rankings)

    # Define array of alternative indices if not provided
    if alt_indices is None:
        alt_indices = range(rankings.shape[1])

    # Define array of rankings names if not provided
    if labels is None:
        labels = [f'$R_{{{i + 1}}}$' for i in range(rankings.shape[0])]
    elif len(labels) != rankings.shape[0]:
        raise ValueError('Length of labels should be equal to number of ranking.')

    # Define default styles for plots
    main_plot_kwargs = dict(
            linewidth=2,
            ) | main_plot_kwargs

    missing_plot_kwargs = dict(
            linewidth=2,
            linestyle='--',
            alpha=0.7,
            zorder=10
            ) | missing_plot_kwargs

    marker_plot_kwargs = dict(
            linestyle=' ',
            marker='o'
            ) | marker_plot_kwargs

    colors = plt.cm.tab20.colors[:16]
    dark_factor = 0.8  # Adjust this factor to control the darkness
    colors = np.clip(np.array(colors) * dark_factor, 0, 1)

    alternative_colors = {}
    # Visualisation is made alternative by alternative.
    for ai in range(rankings.shape[1]):
        # For each alternative we collect points to draw line before and after
        # missing value (0), and also markers
        lines_before = []
        lines_after = []
        markers = []
        # Until we met 0 for this alternative, we collect lines_before,
        # after zero we will collect lines_after
        lines = lines_before

        for i in range(rankings.shape[0]):
            if rankings[i, ai] != 0:
                lines.append((i - space, rankings[i, ai]))
                lines.append((i + space, rankings[i, ai]))
                markers.append((i, rankings[i, ai]))
            else:
                lines = lines_after

        # Add same colors for every object to be drawn
        c = colors[ai]
        main_plot_kwargs['color'] = c
        missing_plot_kwargs['color'] = c
        marker_plot_kwargs['color'] = c

        alternative_colors[ai] = c  # Collect colors for legend

        if lines_before:
            ax.plot(*zip(*lines_before), **main_plot_kwargs)

        if lines_after:
            ax.plot(*zip(*lines_after), **main_plot_kwargs)

        # Draw lines in another style for missing alternatives (if any)
        if lines_before and lines_after:
            ax.plot(*zip(*[lines_before[-1], lines_after[0]]),
                        **missing_plot_kwargs)

        # Add markers with different functions, so we don't have markers on
        # horizontal lines near the vertical ones (space)
        ax.plot(*zip(*markers), **marker_plot_kwargs)

        # Add alternative labels in color of line for this alternative
        ax.text(- space * 1.5,
                rankings[0, ai],
                f'$A_{{{alt_indices[ai] + 1}}}$',
                color=c,
                ha='right', va='center', fontsize='12')
        ax.text(rankings.shape[0] - 1 + space * 1.5,
                rankings[-1, ai],
                f'$A_{{{alt_indices[ai] + 1}}}$',
                color=c,
                ha='left', va='center', fontsize='12')

    ax.set(
        xticks=range(rankings.shape[0]),
        xticklabels=labels,
        xlim=(-0.5, rankings.shape[0] - 0.5),
        ylabel='Position in ranking',
        yticks=range(1, rankings.shape[1] + 1),
        ylim=(0.5, rankings.shape[1] + 0.5)
    )

    if better_grid:
        ax.grid(alpha=0.5, axis='x', linestyle='--',
                linewidth=0.9, color='#CCCCCC')
        for i in range(1, rankings.shape[1] + 1):
            plt.plot([0, rankings.shape[0] - 1], [i, i], linewidth=0.9,
                     linestyle='--', alpha=0.5, zorder=-1, color='#CCCCCC')
    else:
        ax.grid(alpha=0.5, linestyle='--')

    # Create legend with alternative colors
    legend_handles = []
    for alt_index, color in alternative_colors.items():
        legend_handles.append(plt.Line2D([0], [0], marker='s', color='w', markersize=12, markerfacecolor=color, label=f'$A_{{{alt_index+1}}}$ ({country_abbr[alt_index]})'))
    ax.legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc="lower left",
                mode="expand", borderaxespad=0, ncol=8, handles=legend_handles, fontsize=12)


    return ax


Ranking flow visualization generation

In [ ]:
plt.rcParams['figure.figsize'] = (12, 5)
submodel_labels = ['$P_{1_1}$', '$P_{1_2}$', '$P_{2_1}$', '$P_{2_2}$']
ranking_flows(ranks)
plt.xticks([0, 1, 2, 3], submodel_labels, fontsize=14)
y = np.arange(0, matrix.shape[0])
plt.yticks(y+1, y+1, fontsize=14)
plt.xlabel('Submodels', fontsize=18)
plt.ylabel('Ranking position', fontsize=18)
plt.tight_layout()
plt.savefig('img/rankings.pdf')
plt.show()
plt.close()

# Sensitivity analysis

1) Monte Carlo Simulations

In [ ]:
mc_weights = pysensmcda.probabilistic.monte_carlo_weights(9, num_samples=5000)
mc_ranks = []
for matrix_columns in [columns_p11, columns_p12, columns_p21, columns_p22]:
    submodel_ranks = []
    for w in mc_weights:
        matrix = data_df[matrix_columns].to_numpy()
        marcos = MARCOS(normalization_function=pymcdm.normalizations.max_normalization)
        prefs = marcos(matrix, w, criteria_types)
        ranking = marcos.rank(prefs)

        submodel_ranks.append(list(ranking))
    mc_ranks.append(np.array(submodel_ranks))


Visualization of fuzzy ranking based on results from Monte Carlo

In [ ]:
for idx, ranks in enumerate(mc_ranks):
    fuzzy_rank = pysensmcda.ranking.fuzzy_ranking(ranks)
    maxes = np.max(fuzzy_rank, axis=1)
    for idxx, maxx in enumerate(maxes):
        fuzzy_rank[idxx, :] = fuzzy_rank[idxx, :] / maxx
    ax = pysensmcda.graphs.heatmap(fuzzy_rank, figsize=(9, 9))
    ax.figure.axes[-1].yaxis.label.set_size(12)
    plt.xlabel('Alternatives', fontsize=14)
    plt.ylabel('Positions', fontsize=14)
    plt.xticks(y+0.5, [f'$A_{{{i+1}}}$'for i in y], fontsize=12)
    plt.yticks(y+0.5, y+1, fontsize=12)
    plt.title(f'Fuzzy ranking for {submodel_labels[idx]} submodel', fontsize=16)
    plt.savefig(f'img/fuzzy_rank_submodel_{idx+1}.pdf')
    plt.show()
    plt.close()

2) Probabilistic modifications of decision matrix

Method for modification that simulates 5% measurement inaccuracy of data in decision matrix 

In [ ]:
# modified version of method from pysensmcda
# https://github.com/jwieckowski/pysensmcda/blob/main/pysensmcda/probabilistic/perturbed_matrix.py

def perturbed_matrix(matrix: np.ndarray, simulations: int, precision: int = 6, perturbation_scale: float | np.ndarray = 0.1):
    modified_matrices = []

    for _ in range(simulations):
        perturbation = np.random.uniform(-perturbation_scale, perturbation_scale, (matrix.shape[0], matrix.shape[1]))
        modified_matrix = matrix + perturbation
        modified_matrices.append(list(np.round(modified_matrix, precision)))

    return np.array(modified_matrices)

Calculation of results

In [ ]:
probs_ranks = []
for matrix_columns in [columns_p11, columns_p12, columns_p21, columns_p22]:
    matrix = data_df[matrix_columns].to_numpy()
    perturbation = np.mean(matrix) * 0.05
    modified_matrixes = perturbed_matrix(matrix, simulations=1000, perturbation_scale=perturbation)
    
    submodel_ranks = []

    for mod_matrix in modified_matrixes:
        mod_matrix = np.clip(mod_matrix, 0, np.max(mod_matrix))
        marcos = MARCOS(normalization_function=pymcdm.normalizations.max_normalization)
        prefs = marcos(mod_matrix, weights, criteria_types)
        ranking = marcos.rank(prefs)

        submodel_ranks.append(list(ranking))
    probs_ranks.append(np.array(submodel_ranks))


Creation of interval positions in rankings

In [ ]:
interval_ranks = []
for model_ranks in probs_ranks:
    model_interval_rank = []
    for alt in range(matrix.shape[0]):
        model_interval_rank.append([np.min(model_ranks[:, alt]), np.max(model_ranks[:, alt])])
    interval_ranks.append(model_interval_rank)

Visualization of interval results

In [ ]:
colors = plt.cm.tab20.colors[:16]

plt.rcParams['figure.figsize'] = (12, 6)

for idx, model_interval in enumerate(interval_ranks):
    x = np.arange(0, matrix.shape[0])
    for xx, interval in zip(x, model_interval):
        # plt.plot([xx, xx], interval, marker='o', markersize=8, color='dodgerblue', linestyle='--')
        plt.plot([xx, xx], interval, marker='o', markersize=8, color=colors[xx], linestyle='--')
    plt.grid(axis='both', alpha=0.7)
    plt.xticks(x, [f'$A_{{{i+1}}}$' for i in range(matrix.shape[0])], fontsize=16)
    plt.yticks(x+1, x+1, fontsize=16)
    plt.ylabel('Positions', fontsize=18)
    plt.xlabel('Alternatives', fontsize=18)
    plt.title(f'Ranking positions with 5% measurement inaccuracy for {submodel_labels[idx]} submodel', fontsize=18, y=1.22)

    legend_handles = []
    for alt_index, color in enumerate(colors):
        legend_handles.append(plt.Line2D([0], [0], marker='s', color='w', markersize=12, markerfacecolor=color, label=f'$A_{{{alt_index+1}}}$ ({country_abbr[alt_index]})'))
    plt.legend(bbox_to_anchor=(0, 1.02, 1, 0.2), loc="center left",
                mode="expand", borderaxespad=0, ncol=8, handles=legend_handles, fontsize=12)

    plt.tight_layout()
    plt.savefig(f'img/interval_ranks_model_{idx+1}.pdf')
    plt.show()
    plt.close()


3) Criteria relevance identification

In [ ]:
# modified version from pysensmcda
# https://github.com/jwieckowski/pysensmcda/blob/main/pysensmcda/criteria/identification.py

from typing import Union, Callable, List
import pymcdm
from pymcdm.correlations import weighted_spearman
from pymcdm.weights import equal_weights

def relevance_identification(method: callable, call_kwargs: dict, ranking_descending: bool, excluded_criteria: int = 1, corr_coef: Union[Callable, List[Callable]] = weighted_spearman, precision: int = 6) -> list[tuple[tuple[int] | int, tuple[float] | float, float, np.ndarray]]:

    initial_matrix = call_kwargs['matrix'].copy()

    if excluded_criteria > initial_matrix.shape[1]:
        raise ValueError('`excluded_criteria` should not exceed the number of columns in matrix')
    
    types = call_kwargs['types'].copy()

    call_kwargs['weights'] = equal_weights(initial_matrix)

    if not isinstance(corr_coef, list):
        corr_coef = [corr_coef]

    results = []

    excluded = []
    for _ in range(excluded_criteria):
        # remove already excluded criteria
        new_matrix = np.delete(initial_matrix, excluded, axis=1)
        new_weights = equal_weights(new_matrix)
        new_types = np.delete(types, excluded, axis=0)

        # update call parameters
        call_kwargs['matrix'] = new_matrix
        call_kwargs['weights'] = new_weights
        call_kwargs['types'] = new_types 

        # calculate the initial evaluation
        try:
            ref_preferences = method(**call_kwargs)
            ref_ranking = pymcdm.helpers.rankdata(ref_preferences, ranking_descending)
        except Exception as err:
            raise ValueError(err)

        # index of minimum change
        min_change_idx = None
        min_distance = 1 * new_matrix.shape[0]

        temp_results = []
        for i in range(initial_matrix.shape[1]):
            if i in excluded:
                continue
            index = i - len([e_idx for e_idx in excluded if e_idx < i]) if any([i > e_idx for e_idx in excluded]) else i

            # modify input data
            modified_matrix = np.delete(new_matrix, index, axis=1)
            modified_weights = equal_weights(modified_matrix)
            modified_types = np.delete(new_types, index, axis=0)

            # update call parameters
            call_kwargs['matrix'] = modified_matrix
            call_kwargs['weights'] = modified_weights
            call_kwargs['types'] = modified_types 

            # calculate results
            new_preferences = method(**call_kwargs)
            new_ranking = pymcdm.helpers.rankdata(new_preferences, ranking_descending)
            corr_results = []
            for corr in corr_coef:
                corr_results.append(np.round(corr(ref_ranking, new_ranking), precision))
            distance = np.round(np.sum(np.sqrt((ref_preferences - new_preferences)**2)), precision)

            temp_results.append((tuple(excluded + [i]), *corr_results, distance, modified_matrix))

        excluded.append(min_change_idx)
        results.append(temp_results)

    return results

Visualization method for results from criteria relevance identification

In [ ]:
# modified version from pymcdm
# https://gitlab.com/shekhand/mcda/-/blob/master/pymcdm/visuals/rankings_flow_correlation.py

from mpl_toolkits.axes_grid1 import make_axes_locatable
from pymcdm.visuals import ranking_bar
from pymcdm.visuals import correlation_plot

def rankings_flow_correlation(rankings,
                              correlations,
                              labels=None,
                              correlation_name='Correlation',
                              correlation_ax_size='25%',
                              correlation_plot_kwargs=dict(),
                              ranking_flows_kwargs=dict(),
                              ax=None,
                              my_labels=None):

    if ax is None:
        ax = plt.gca()

    divider = make_axes_locatable(ax)
    cax = divider.append_axes("top", size=correlation_ax_size, pad=0.05)

    correlation_plot(correlations, ylabel=correlation_name, labels=labels, **correlation_plot_kwargs, ax=cax, ylim=[0.91, 1])
    cax.tick_params(bottom=False, labelbottom=False)
    cax.figure.axes[-1].yaxis.label.set_size(14)

    x = np.arange(0, matrix.shape[1])
    ax.bar(x, rankings, width=0.7, alpha=0.9, color='dodgerblue', linewidth=1, edgecolor='darkblue')
    ax.grid(axis='both', alpha=0.7)
    ax.set_xticks(x)
    ax.set_xticklabels(my_labels, fontsize=14)
    ax.set_ylabel('Sum of preference score differences', fontsize=14)

    for xx, yy in zip(x, rankings):
        ax.text(x=xx-0.22, y=yy+0.01, s='%.3f' % yy, fontsize=12)

    ax.set_ylim([0, max(rankings)+0.07])
    cax.set_xlim(ax.get_xlim())

    return ax, cax


Results for $r_W$ correlation coefficient 

In [ ]:
plt.rcParams['figure.figsize'] = (15, 6)
for idx, matrix_columns in enumerate([columns_p11, columns_p12, columns_p21, columns_p22]):
    matrix = data_df[matrix_columns].to_numpy()
    call_kwargs = {
        "matrix": matrix,
        "weights": weights,
        "types": criteria_types,
    }

    results = relevance_identification(marcos, call_kwargs, ranking_descending=True)

    x = np.arange(0, matrix.shape[1])
    y = [r[1] for r in results[0]]
    y_diff = [r[2] for r in results[0]]

    plt.rcParams['axes.axisbelow'] = True
    rankings_flow_correlation(np.array(y_diff), np.array(y), correlation_name='$r_W$', my_labels=[f'w/o $C_{{{i+1}}}$' for i in x])
    plt.savefig(f'img/criteria_relevance_model_{idx+1}.pdf')
    plt.show()
    plt.close()


Results for $WS$ correlation coefficient 

In [ ]:
for idx, matrix_columns in enumerate([columns_p11, columns_p12, columns_p21, columns_p22]):
    matrix = data_df[matrix_columns].to_numpy()
    call_kwargs = {
        "matrix": matrix,
        "weights": weights,
        "types": criteria_types,
    }

    results = relevance_identification(marcos, call_kwargs, ranking_descending=True, corr_coef=pymcdm.correlations.ws)

    x = np.arange(0, matrix.shape[1])
    y = [r[1] for r in results[0]]
    y_diff = [r[2] for r in results[0]]

    plt.rcParams['axes.axisbelow'] = True
    rankings_flow_correlation(np.array(y_diff), np.array(y), correlation_name='$WS$', my_labels=[f'w/o $C_{{{i+1}}}$' for i in x])
    plt.savefig(f'img/criteria_relevance_ws_model_{idx+1}.pdf')
    plt.show()
    plt.close()


4) Ranking demotion calculation

Setup of positions to which alternatives should be demoted

In [ ]:
demoted_initial_ranks = initial_ranks + 1
demoted_initial_ranks = np.clip(demoted_initial_ranks, 1, 16)

Results calculation

In [ ]:
all_results = []
matrices = []
for idx, matrix_columns in enumerate([columns_p11, columns_p12, columns_p21, columns_p22]):

    matrix = data_df[matrix_columns].to_numpy()
    matrices.append(matrix)

    bounds = []
    for i in range(matrix.shape[1]):
        if criteria_types[i] == 1:
            bounds.append(0)
        else:
            bounds.append(np.max(matrix[:, i]))
    bounds = np.array(bounds)

    call_kwargs = {
        "matrix": matrix,
        "weights": weights,
        "types": criteria_types,
    }
    temp_res = pysensmcda.ranking.ranking_demotion(matrix, initial_ranks[idx], marcos, call_kwargs, ranking_descending=True, direction=criteria_types*-1, step=1, positions=demoted_initial_ranks[idx], bounds=bounds)
    all_results.append(temp_res)
    

Visualization method

In [ ]:
# modified version from pysensmcda
# https://github.com/jwieckowski/pysensmcda/blob/main/pysensmcda/graphs/pd_ranking_graph.py

import matplotlib.ticker as mtick

def percentage_graph(percentage_changes: list | np.ndarray, 
                    new_positions: list, 
                    ax: plt.Axes | None = None, 
                    xticks: list | None = None, 
                    percentage_kwargs: dict = dict(), 
                    kind: str = 'bar', 
                    palette: dict = dict()) -> plt.Axes:

    if ax is None:
        ax = plt.gca()

    crit_num = len(percentage_changes)
    min_change = np.round(np.min(percentage_changes))
    max_change = np.round(np.max(percentage_changes))
    step = int(np.max(np.abs([min_change/5, max_change/5])))

    ax.grid(axis='y', alpha=0.5, linestyle='--')
    ax.set_axisbelow(True)

    ax.plot([-1, len(percentage_changes)], [0, 0], percentage_kwargs.get('base_linestyle', '--'), color=palette.get('neutral', 'black'), linewidth=percentage_kwargs.get('base_linewidth', 1))

    if kind == 'bar':
        colors = [palette.get('positive', 'blue') if change > 0  else palette.get('negative', 'red') for change in percentage_changes]
        p = ax.bar(np.arange(0, crit_num), percentage_changes, color=colors)
        if percentage_kwargs.get('show_percenatage_value', True):
            ax.bar_label(p, label_type='center', fmt=lambda x: '' if x == 0 else f'{x:.0f} %')
        if percentage_kwargs.get('show_ranks', True):
            ax.bar_label(p, labels=[f'Rank {int(rank)}' for rank in new_positions])
        for idx, change in enumerate(percentage_changes):
            if change == 0:
                ax.plot([idx-0.4, idx+0.4], [0, 0], color=palette.get('neutral', 'black'))
    elif kind == 'line':
        for idx, change in enumerate(percentage_changes):
            if change > 0:
                color = palette.get('positive', 'blue')
                marker = percentage_kwargs.get('positive_marker', '^')
                marker_size = percentage_kwargs.get('marker_size', percentage_kwargs.get('positive_markersize', 8))
            elif change < 0:
                color = palette.get('negative', 'red')
                marker = percentage_kwargs.get('negative_marker', 'v')
                marker_size = percentage_kwargs.get('marker_size', percentage_kwargs.get('negative_markersize', 8))
            else:
                color = palette.get('neutral', 'black')
                marker = percentage_kwargs.get('neutral_marker', 'o')
                marker_size = percentage_kwargs.get('marker_size', percentage_kwargs.get('neutral_markersize', 8))
            ax.plot([idx], [change], color=color, marker=marker, markersize=marker_size)
            ax.plot([idx, idx], [0, change], percentage_kwargs.get('linestyle', '-'), linewidth=percentage_kwargs.get('linewidth', 3), color=color)
        if percentage_kwargs.get('show_ranks', True):
            for idx, change in enumerate(percentage_changes):
                dist = np.sign(change)*step/2 if np.sign(change) else step/2
                ax.text(x=idx , y=change+dist, s=f'Rank {int(new_positions[idx])}', ha='center')
    if not np.all(percentage_changes == 0):
        ax.set_ylim(np.min(percentage_changes) - step, np.max(percentage_changes) + step)
    ax.set_xlim(-0.5, crit_num-0.5)
    
    if xticks is None:
        ax.set_xticks(np.arange(0, crit_num), [f'$C_{{{i+1}}}$' for i in range(crit_num)], fontsize=14)
    else:
        ax.set_xticks(np.arange(0, crit_num), xticks)
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    ax.set_ylabel(percentage_kwargs.get('ylabel', ''), fontsize=14)
    ax.set_title(percentage_kwargs.get('title', ''))

    return ax

def rank_graph(initial_rank: int | float, 
            new_positions: list, 
            ax: plt.Axes | None = None, 
            palette: dict = dict(), 
            rank_kwargs: dict = dict()) -> plt.Axes:

    if ax is None:
        ax = plt.gca()

    ax.grid(axis='y', linestyle='--', alpha=0.5)
    ax.set_axisbelow(True)
    
    ax.plot([-1, len(new_positions)], [initial_rank, initial_rank], rank_kwargs.get('base_linestyle', '--'), color=palette.get('neutral', 'black'), linewidth=rank_kwargs.get('base_linewidth', 1))

    for idx, rank in enumerate(new_positions):
        if rank < initial_rank:
            color = palette.get('positive', 'blue')
        elif rank > initial_rank:
            color = palette.get('negative', 'red')
        else:
            color = palette.get('neutral', 'black')
        ax.plot([idx, idx], [initial_rank, rank], rank_kwargs.get('linestyle', '--'), color=color)
        ax.plot(idx, rank, rank_kwargs.get('marker', '*'), color=color, markersize=rank_kwargs.get('markersize', 10))
    
    ax.set_yticks(np.arange(np.min([initial_rank, *new_positions]), np.max([initial_rank, *new_positions])+1))
    ax.set_ylim(np.min([initial_rank, *new_positions])-0.5, np.max([initial_rank, *new_positions])+0.5)
    ax.invert_yaxis()
    ax.set_ylabel(rank_kwargs.get('ylabel', ''), fontsize=14)
    ax.set_title(rank_kwargs.get('title', ''))

    return ax

def pd_rankings_graph(initial_rank: int | float, 
                    new_positions: list, 
                    percentage_changes: list | np.ndarray, 
                    xticks: list | None = None, 
                    kind: str = 'bar', 
                    title: str = '', 
                    ax: plt.Axes | None = None, 
                    draw_ranking_change: bool = True, 
                    height_ratio: list[int] = [1, 3], 
                    percentage_kwargs: dict = dict(), 
                    rank_kwargs: dict = dict(), 
                    palette: dict = dict()) -> plt.Axes:

    if not palette:
        palette = {
            'positive': '#4c72b0',
            'neutral': 'black',
            'negative': '#c44e52',
        }
    if not rank_kwargs:
        rank_kwargs = {
            'ylabel': 'Rank change',
        }

    if not percentage_kwargs:
        percentage_kwargs = {
            'ylabel': 'Criterion value change'
        }
                
    if ax is None:
        if draw_ranking_change:
            fig, (cax, main_ax) = plt.subplots(2, 1, gridspec_kw={'height_ratios': height_ratio})
            cax.tick_params(bottom=False, labelbottom=False)
        else:
            fig, main_ax = plt.subplots()
    else:
        if draw_ranking_change:
            try:
                (cax, main_ax) = ax
            except TypeError:
                raise TypeError("If 'draw_ranking_change'='True', the 'ax' parameter needs to consist of two axes")
        else:
            main_ax = ax

    percentage_graph(percentage_changes, new_positions, xticks=xticks, percentage_kwargs=percentage_kwargs, palette=palette, ax=main_ax, kind=kind)

    if draw_ranking_change:
        rank_graph(initial_rank, new_positions, palette=palette, rank_kwargs=rank_kwargs, ax=cax)
        cax.set_xlim(main_ax.get_xlim())

    fig.align_ylabels()
    plt.suptitle(title, fontsize=16)
    
    return (cax, main_ax)

Results visualizations

In [ ]:
for idx, results in enumerate(all_results):
    results = np.array(results)
    alt_num = 16
    crit_num = 9
    init_rank = initial_ranks[idx]
    matrix = matrices[idx]
    for alt in range(alt_num):
        alt_results = results[results[:, 0] == alt]
        percentage_changes = []
        new_positions = []
        if len(alt_results):
            for crit in range(crit_num):
                r = alt_results[alt_results[:, 1] == crit]
                if len(r):
                    _ , crit, change, new_pos = r[0]
                    crit = int(crit)
                    if init_rank[alt] == new_pos:
                        percentage_changes.append(0)
                        new_positions.append(init_rank[alt])
                    else:
                        if change == 0 or matrix[alt, crit] == 0:
                            percentage_changes.append(0)
                            new_positions.append(init_rank[alt])
                        else:
                            percentage_changes.append((change - matrix[alt, crit])/matrix[alt, crit]*100)
                            new_positions.append(new_pos)
                    
                else:
                    percentage_changes.append(0)
                    new_positions.append(init_rank[alt])
            
            pd_rankings_graph(init_rank[alt], new_positions, np.array(percentage_changes), kind='bar', title=f'Ranking demotion for {submodel_labels[idx]} submodel - {country_names[alt]} ($A_{{{alt+1}}}$)')
            plt.savefig(f'img/ranking_demotion_model_{idx+1}_a{alt+1}.pdf')
            plt.show()
            plt.close()